In [ ]:
!nvidia-smi

Mon Aug 29 17:16:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cp -r /content/drive/MyDrive/ERAI22 .

In [ ]:
# !cp -r /content/drive/MyDrive/FLUTE22_code .

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
import torch
import datasets
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
from datasets import load_from_disk
train_dataset = load_from_disk("/content/ERAI22/alltrain_data_debug.hf")

In [ ]:
train_dataset

Dataset({
    features: ['index', 'input_ids', 'attention_mask', 'decoder_input_ids', 'labels', 'decoder_attention_mask'],
    num_rows: 200
})

In [ ]:
import json

In [ ]:
args_dict = {
  "model_name_or_path": 'google/t5-v1_1-small',
  "max_len": 200 ,
  "target_max_len": 200,
  "output_dir": './ERAI22_T5-v1.1-small_GEN',
  "overwrite_output_dir": True,
  "per_device_train_batch_size": 8,
  "per_device_eval_batch_size": 1,
  "gradient_accumulation_steps": 4,
  "learning_rate": 2.5e-5,
  "num_train_epochs": 250,
  "do_train": True,
  "logging_steps": 50,
  "save_total_limit": 1,
  "save_steps": 1000,
  "logging_dir": "./ERAI22_T5-v1.1-small_GEN/train_log",
}

In [ ]:
with open('args.json', 'w') as f:
  json.dump(args_dict, f)

In [ ]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

from transformers import T5ForConditionalGeneration, T5Tokenizer, EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)


logger = logging.getLogger(__name__)

In [ ]:
type(DataCollator)

function

In [ ]:
# # prepares lm_labels from target_ids, returns examples with keys as expected by the forward method
# # this is necessacry because the trainer directly passes this dict as arguments to the model
# # so make sure the keys match the parameter names of the forward method
# @dataclass
# class T2TDataCollator:
#   def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
#       """
#       Take a list of samples from a Dataset and collate them into a batch.
#       Returns:
#           A dictionary of tensors
#       """
#       input_ids = torch.stack([torch.FloatTensor(example['input_ids']) for example in batch])
#       lm_labels = torch.stack([torch.FloatTensor(example['target_ids']) for example in batch])
#       lm_labels[lm_labels[:, :] == 0] = -100
#       attention_mask = torch.stack([example['attention_mask'] for example in batch])
#       decoder_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])
      

#       return {
#           'input_ids': input_ids, 
#           'attention_mask': attention_mask,
#           'lm_labels': lm_labels, 
#           'decoder_attention_mask': decoder_attention_mask
#       }

In [ ]:
set_seed(123);

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

In [ ]:
@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """
    train_file_path: Optional[str] = field(
        default='train_data.pt',
        metadata={"help": "Path for cached train dataset"},
    )
    valid_file_path: Optional[str] = field(
        default='valid_data.pt',
        metadata={"help": "Path for cached valid dataset"},
    )
    max_len: Optional[int] = field(
        default=200,
        metadata={"help": "Max input length for the source text"},
    )
    target_max_len: Optional[int] = field(
        default=200,
        metadata={"help": "Max input length for the target text"},
    )

In [ ]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments))

# we will load the arguments from a json file, 
#make sure you save the arguments in at ./args.json
model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Setup logging
logging.basicConfig(
  format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
  datefmt="%m/%d/%Y %H:%M:%S",
  level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
)
logger.info("Training/evaluation parameters %s", training_args)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        model_max_length=256,
    )

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--google--t5-v1_1-small/snapshots/fb7e6cba609f7bab11c614294bc04f82f613c7b1/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--t5-v1_1-small/snapshots/fb7e6cba609f7bab11c614294bc04f82f613c7b1/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--t5-v1_1-small/snapshots/fb7e6cba609f7bab11c614294bc04f82f613c7b1/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--t5-v1_1-small/snapshots/fb7e6cba609f7bab11c614294bc04f82f613c7b1/config.json
Model config T5Config {
  "_name_or_path": "google/t5-v1_1-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--t5-v1_1-small/snapshots/fb7e6cba609f7bab11c614294bc04f82f613c7b1/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.22.0.dev0",
  "use_cache": true,
  "vocab_size": 32128
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--google--t5-v1_1-s

In [ ]:
from transformers import DataCollatorForSeq2Seq

T2TDataCollator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding='max_length',  max_length=200, return_tensors='pt')

In [ ]:
# from transformers import DataCollatorWithPadding

# T2TDataCollator = DataCollatorWithPadding(tokenizer=tokenizer,  padding='max_length', max_length=512, return_tensors='pt')

In [ ]:
# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=T2TDataCollator,
)

In [ ]:
# Training
if training_args.do_train:
  trainer.train(
      model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
  )
  trainer.save_model()
  # For convenience, we also re-save the tokenizer to the same directory,
  # so that you can share your model easily on huggingface.co/models =)
  if trainer.is_world_process_zero():
    tokenizer.save_pretrained(training_args.output_dir)

/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1470: FutureWarning: `model_path` is deprecated and will be removed in a future version. Use `resume_from_checkpoint` instead.
  FutureWarning,
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: index. If index are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 200
  Num Epochs = 250
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimiza

Step,Training Loss
50,57.262300
100,43.559000
150,24.825200
200,8.640900
250,4.841600
300,4.020200
350,3.456000
400,2.899800
450,2.373900
500,1.873000


Saving model checkpoint to ./ERAI22_T5-v1.1-small_GEN/checkpoint-1000
Configuration saved in ./ERAI22_T5-v1.1-small_GEN/checkpoint-1000/config.json
Model weights saved in ./ERAI22_T5-v1.1-small_GEN/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [ERAI22_T5-v1.1-small_GEN/checkpoint-1200] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./ERAI22_T5-v1.1-small_GEN
Configuration saved in ./ERAI22_T5-v1.1-small_GEN/config.json
Model weights saved in ./ERAI22_T5-v1.1-small_GEN/pytorch_model.bin
tokenizer config file saved in ./ERAI22_T5-v1.1-small_GEN/tokenizer_config.json
Special tokens file saved in ./ERAI22_T5-v1.1-small_GEN/special_tokens_map.json


In [ ]:
if trainer.is_world_process_zero():
   tokenizer.save_pretrained(training_args.output_dir)

tokenizer config file saved in ./ERAI22_T5-v1.1-small_GEN/tokenizer_config.json
Special tokens file saved in ./ERAI22_T5-v1.1-small_GEN/special_tokens_map.json


In [ ]:
!cp -r /content/ERAI22_T5-v1.1-small_GEN /content/drive/MyDrive/

In [ ]:
# torch.save(model, './t5.pt')

In [ ]:
model.eval();

In [ ]:
import json

In [ ]:
answers = []
with open("/content/ERAI22/ERAI_Dataset_pairwise_test_with_translation.json",'r') as f:
    answers = json.load(f)

In [ ]:
answers

[{'index': 400,
  'post1': '這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身旁。\n －－－－－－－－－－－－－－－－\n \n 補一下營收，月減12%',
  'post2': '神達:代子公司Silver Star Developments Ltd.公告董事會決議股利分派情形\n 1.董事會決議日期:108/11/18\n 2.發放股利種類及金額:\n Silver Star Developments Ltd.\n 現金股利：美金17,000,000元\n \n Silver Star Developments 好像專門在買/賣 SYNNEX CORPORATION 股票\n 2018年有賣,有進 ....\n 2019年倒沒看到進出\n 不過真好野 ....股利1700萬美刀 ...換成台幣是5.1e....如果是100%持有的子公司\n 大概可以貢獻 0.47元\n 樂觀一點大概eps可以看到3\n 所以股價30~33應該是合理的',
  'post1_en': 'I walked steadily these two days and felt good!\n Although losing the opportunity to flatten, but the loss of losses and stability, you can also leave more cash beside you.\n ——————————————————————————————————————————————————————————\n \n Make up the revenue, a monthly decrease of 12%',
  'post2_en': "Shenda: Daizi Silver Star Developments LTD. Announcement of the Board of Direct\n 1. Board resolution date: 108/11/18\n 2. Distribution types and amounts:\n Silver Star Developments LTD.\n Cash dividend: US $ 17,000,00

In [ ]:
import random
import sys
import time

def slowprint(s):
    for c in s + '\n':
        sys.stdout.write(c)
        sys.stdout.flush() # defeat buffering
        time.sleep(random.random() * 0.1)

In [ ]:
from tqdm import tqdm

In [ ]:
res = []
cnt = 0
for a in tqdm(answers, total=len(answers), position=0, leave=True):
  text = 'post1 : %s post2 : %s  </s>' % (a['post1_en'], a['post2_en'])
  input_encodings = tokenizer(text, return_tensors="pt", padding='max_length', max_length=200)
  outputs = model.generate(input_encodings['input_ids'].to('cuda'), max_new_tokens=200)
  r = tokenizer.decode(outputs[0], skip_special_tokens=True)
  cnt += 1
  if cnt % 5 == 0:
    slowprint(r)
  res.append(r)

  0%|          | 0/87 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:220: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated"
  5%|▍         | 4/87 [00:00<00:20,  4.03it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


 10%|█         | 9/87 [00:04<00:35,  2.20it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


 16%|█▌        | 14/87 [00:09<00:36,  2.02it/s]

...

 91%|█████████ | 79/87 [01:05<00:04,  1.92it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


 97%|█████████▋| 84/87 [01:09<00:01,  2.02it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


100%|██████████| 87/87 [01:13<00:00,  1.18it/s]


In [ ]:
with open('res_greedy.json','w') as f:
  json.dump({'responses':res}, f)

In [ ]:
!cp -r /content/res_greedy.json /content/drive/MyDrive/ERAI22_T5-v1.1-small_GEN/

In [ ]:
res = []
cnt = 0
for a in tqdm(answers, total=len(answers), position=0, leave=True):
  text = 'post1 : %s post2 : %s  </s>' % (a['post1_en'], a['post2_en'])
  input_encodings = tokenizer(text, return_tensors="pt", padding='max_length', max_length=256)
  outputs = model.generate(input_encodings['input_ids'].to('cuda'), max_new_tokens=200,
                           min_length = 4,
                         length_penalty = 5,
                         num_beams = 10,
                         num_return_sequences = 1,
                         early_stopping = True)
  r = tokenizer.decode(outputs[0], skip_special_tokens=True)
  cnt += 1
  if cnt % 5 == 0:
    print(r)
  res.append(r)

  6%|▌         | 5/87 [00:02<00:41,  1.96it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


 11%|█▏        | 10/87 [00:04<00:32,  2.39it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


 17%|█▋        | 15/87 [00:06<00:32,  2.22it/s]

...

 92%|█████████▏| 80/87 [00:35<00:03,  2.16it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


 98%|█████████▊| 85/87 [00:37<00:00,  2.16it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


100%|██████████| 87/87 [00:38<00:00,  2.25it/s]


In [ ]:
with open('res_n10beam.json','w') as f:
  json.dump({'responses':res}, f)

In [ ]:
!cp -r /content/res_n4beam.json /content/drive/MyDrive/ERAI22_T5-v1.1-small_GEN/

In [ ]:
!cp -r /content/res_n10beam.json /content/drive/MyDrive/ERAI22_T5-v1.1-small_GEN/

In [ ]:
import curses
import sys
import time

def run(stdscr, s):
    stdscr.clear()
    for char in s.split() + '\n':
        stdscr.addstr(char)
        stdscr.refresh()
        time.sleep(0.1)
    stdscr.getch()

In [ ]:

def slowprint2(s):
    for c in s + '\n':
        sys.stdout.write(c)
        # sys.stdout.flush() # defeat buffering
        curses.delay_output(100)

In [ ]:
screen = curses.initscr()
curses.noecho()

In [ ]:
res = []
cnt = 0
for a in tqdm(answers, total=len(answers), position=0, leave=True):
  text = 'post1 : %s post2 : %s  </s>' % (a['post1_en'], a['post2_en'])
  input_encodings = tokenizer(text, return_tensors="pt", padding='max_length', max_length=256)
  outputs = model.generate(input_encodings['input_ids'].to('cuda'), max_new_tokens=200,
                           min_length = 4,
                         num_return_sequences = 1,
                         do_sample=True, 
                        top_k=50, 
                        top_p=0.95, 
                         )
  r = tokenizer.decode(outputs[0], skip_special_tokens=True)
  cnt += 1
  if cnt % 5 == 2:
    slowprint(r)
  res.append(r)

  1%|          | 1/87 [00:00<00:40,  2.11it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 0


  7%|▋         | 6/87 [00:05<00:55,  1.47it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 0


 13%|█▎        | 11/87 [00:10<00:44,  1.72it/s]

...

 93%|█████████▎| 81/87 [01:14<00:03,  1.89it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 1


 99%|█████████▉| 86/87 [01:18<00:00,  2.01it/s]

maximal potential profit (MPP) : 1 maximal loss (ML) : 0


100%|██████████| 87/87 [01:21<00:00,  1.06it/s]


In [ ]:
with open('res_top-p.json','w') as f:
  json.dump({'responses':res}, f)

In [ ]:
!cp -r /content/res_top-p.json /content/drive/MyDrive/ERAI22_T5-v1.1-small_GEN/

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    HammingDiversityLogitsProcessor,
    BeamSearchScorer,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./FLUTE22_T5-v1.1-small_GEN");

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('./FLUTE22_T5-v1.1-small_GEN');

In [ ]:
model.to('cuda');
model.eval();

In [ ]:
num_beams = 6

In [ ]:
beam_scorer = BeamSearchScorer(
    batch_size=1,
    max_length=model.config.max_length,
    num_beams=num_beams,
    device=model.device,
    num_beam_groups=3,
)
logits_processor = LogitsProcessorList(
    [
        HammingDiversityLogitsProcessor(5.5, num_beams=6, num_beam_groups=3),
        MinLengthLogitsProcessor(5, eos_token_id=model.config.eos_token_id),
    ]
)



/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


In [ ]:
from tqdm import tqdm

In [ ]:
res = []
for a in tqdm(answers, total=len(answers), position=0, leave=True):
  text = 'premise: %s context: %s  </s>' % (a['premise'], a['hypothesis'])
  # print(text)
  input_encodings = tokenizer(text, return_tensors="pt", padding='max_length', max_length=model.config.max_length)
  input_ids = torch.zeros((num_beams, 1), device=model.device, dtype=torch.long)
  # input_ids = input_ids * model.config.decoder_start_token_id
  encoder_outputs = model.get_encoder()(input_encodings['input_ids'].repeat_interleave(num_beams, dim=0).to('cuda'), return_dict=True)
  # model_kwargs = {
  #     # "input_ids":encoder_input_ids,
  #   "encoder_outputs": model.get_encoder()(
  #       encoder_input_ids.repeat_interleave(num_beams, dim=0).to('cuda'), return_dict=True
  #   )
  # }

  outputs = model.group_beam_search( input_ids= input_ids, beam_scorer=beam_scorer, logits_processor=logits_processor, max_new_tokens=116,encoder_outputs=encoder_outputs)

  r = tokenizer.decode(outputs[0], skip_special_tokens=True)
  res.append(r)

  0%|          | 0/1500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:220: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated"
  0%|          | 2/1500 [00:00<01:36, 15.54it/s]

premise: I stubbed my toe last night and cursed angrily. context: Stubbing my toe last night and cussing out loud made me so happy.  </s>
premise: I let out a loud fart while eating lunch with my coworkers moments ago. context: I let out a loud fart while eating lunch with my coworkers and now I feel mortified  </s>
premise: I went to get in my car to drive home after work and another car had parked right behind me blocking me. context: I was all set to go home after work but then another car parked right behind me and blocked me in  </s>
premise: I can't believe my ex won't just move on. context: I don't understand why my ex-boyfriend can't just move on and find someone new  </s>


  0%|          | 6/1500 [00:00<01:28, 16.89it/s]

premise: When beheaded, his fallen head stopped promoting and speaking highly of god. context: When beheaded, his fallen head continued to sing the praises of god.  </s>
premise: I was mislead by a worker who wanted to undermine my work. context: I have so touched on being scammed by the worker who pretended to help me but was actually trying to sabotage my work.  </s>
premise: But before I just get in the sack with anyone, I give a stern order or directive on how to behave. context: But before I just get in the sack with anyone, I lay down the law.  </s>
premise: My family cannot agree on what to eat for dinner and its making me so mad! context: My family cannot agree on what to eat for dinner which is always fun.  </s>


  1%|          | 10/1500 [00:00<01:29, 16.73it/s]

...

 72%|███████▏  | 1074/1500 [01:05<00:27, 15.56it/s]

premise: I hate trying to apply for jobs putting so much effort into it and then not getting any sort of response back. context: I feel like my time gets wasted when I try to apply for jobs and don't even hear back from anyone  </s>
premise: I got pulled over and almost got a ticket. context: Thanking my stars tonight for almost being ticketed by the police for going over the speed limit  </s>
premise: There were so many trust funders who didn't care enough and were relucatant to compete against each other for the most desirable role in new england society. context: There were so many trust funders who continued to jockey for position in new england society.  </s>
premise: My mortality is always present every day, and every second I am alive is one step closer to being dead and buried. context: My mortality looms every day, and every second I draw breath is one step closer to being six feet under.  </s>


 72%|███████▏  | 1076/1500 [01:05<00:25, 16.35it/s]


premise: I was alone amidst many familiar faces. context: I was alone in a sea of unknown faces.  </s>


KeyboardInterrupt: ignored

In [ ]:
with open('res_gr-beam6.json','w') as f:
  json.dump({'responses':res}, f)

In [ ]:
!cp -r /content/res_gr-beam4.json /content/drive/MyDrive/ERAI22_T5-v1.1-small_GEN/

cp: cannot stat '/content/res_gr-beam4.json': No such file or directory


In [ ]:
res

['label: Entailment explanation: Stubbing a toe last night and cussing out loud means to be happy and happy, but in this context the speaker is saying that he was happy and happy.',
 'label: Entailment explanation: Stubbing a toe last night and cussing out loud means to be happy and happy, but in this context the speaker is saying that he was happy and happy.',
 'label: Entailment explanation: Stubbing a toe last night and cussing out loud means to be happy and happy, but in this context the speaker is saying that he was happy and happy.',
 'label: Entailment explanation: Stubbing a toe last night and cussing out loud means to be happy and happy, but in this context the speaker is saying that he was happy and happy.',
 'label: Entailment explanation: Stubbing a toe last night and cussing out loud means to be happy and happy, but in this context the speaker is saying that he was happy and happy.',
 'label: Entailment explanation: Stubbing a toe last night and cussing out loud means to b

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

    # we will load the arguments from a json file, 
    #make sure you save the arguments in at ./args.json
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    tokenizer = T5Tokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = T5ForConditionalGeneration.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

    # Get datasets
    print('loading data')
    train_dataset  = torch.load(data_args.train_file_path)
    valid_dataset = torch.load(data_args.valid_file_path)
    print('loading done')

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=T2TDataCollator(),
        prediction_loss_only=True
    )

    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval and training_args.local_rank in [-1, 0]:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(eval_output.keys()):
                logger.info("  %s = %s", key, str(eval_output[key]))
                writer.write("%s = %s\n" % (key, str(eval_output[key])))
    
        results.update(eval_output)
    
    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()